# MTH3302 - Méthodes probabilistes et statistiques pour I.A.
#### Polytechnique Montréal
### Projet A2024
----
Équipe T - TODO
### Objectif
Prédiction de **la consommation en carburant de voitures récentes**.

### Données
Le jeu de données contient pour presque 400 véhicule, la consommation moyenne en L/100km, l'année de frabrication, le type de véhicule, le nombre de cylindre, cylindree, la transmission et la boite.

- `train.csv` est l'ensemble d'entraînement
- `test.csv` est l'ensemble de test


## Chargement des données

Importation des librairies utilisées dans le calepin.

// TODO: à enlever à la fin:

Pour importer librairies:
using Pkg
Pkg.add(["CSV", "DataFrames", "Combinatorics", "Gadfly", "Distributions"], ...)

In [464]:
# import Pkg; Pkg.add("Plots")

In [465]:
# using CSV, DataFrames, Statistics, Dates, Gadfly, LinearAlgebra, Plots

In [466]:
function standardize(data)
    return (data .- mean(data)) ./ std(data)
end

function standardize_data(data)
    stddata = deepcopy(data)
   for col in names(stddata)
    
        print(col)
        if eltype(stddata[!, col]) <: Number && col != "id"
            stddata[!, col] = standardize(stddata[!, col])
        end
    end
    return stddata
end

standardize_data (generic function with 1 method)

In [467]:
function encode(data, column)
    for c in unique(data[!, column])
        data[!, Symbol(c)] = ifelse.(data[!, column] .== c, 1, 0)
    end
    return data
end

function encode_data(data)
    encoded_data = deepcopy(data)
    encoded_data = encode(encoded_data, :type)
    encoded_data = encode(encoded_data, :transmission)
    encoded_data = encode(encoded_data, :boite)
    return encoded_data
end

encode_data (generic function with 1 method)

In [468]:
# faire une méthode pour les outliers

# code pas tester:
# function remove_outliers(data)
#     return data[(abs.(zscore(data)) .< 3) .| isnan.(zscore(data)), :]
# end

In [469]:
function removeRows(data)
    return select!(data, Not([:type, :transmission, :boite]))
    # return select!(data, Not([:annee]))
end

removeRows (generic function with 1 method)

In [470]:
function getStandardEncodedData(data)
    data_copy = deepcopy(data)
    standardised_data = standardize_data(data_copy)
    standardised_data = encode_data(standardised_data,)
    standardised_data = removeRows(standardised_data)
    
    return standardised_data
end




getStandardEncodedData (generic function with 1 method)

In [471]:
function rmse(y, ychap)
    return sqrt(mean((ychap .- y).^2))
end


rmse (generic function with 1 method)

In [472]:
function rsquared(y, ychap)
    ss_total = sum((y .- mean(y)).^2)
    ss_res = sum((y .- ychap).^2)
    return ss_res / ss_total
end


rsquared (generic function with 1 method)

In [473]:
function split_data(data,k)
    n = size(data, 1)
    validation_size = floor(Int, n * k)

    validation_indices = rand(1:n, validation_size)  
    
    validation_data = data[validation_indices, :]
    training_data = data[setdiff(1:n, validation_indices), :]
    
    return training_data, validation_data
end

split_data (generic function with 1 method)

In [474]:
function regression(standardised_data, trainTestSplitPercentage)
    training_data, test_data = split_data(standardised_data, trainTestSplitPercentage)
    X_train =  Matrix(training_data[:, Not(:consommation, :id)])
    y_train = training_data[:, :consommation]

    beta = X_train \ y_train

    rmseval = 0.0
    if trainTestSplitPercentage != 0.0
        X_test = Matrix(test_data[:, Not(:consommation, :id)])
        y_test = test_data[:, :consommation]
        ychap =  X_test * beta
        ychap = (ychap .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        y_test = (y_test .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        # print("regression\n")
        # print("r2 = ", rsquared(y_test, ychap), "\n")
        rmseval = rmse(y_test, ychap)
        # print("rmse = ", rmseval , "\n\n")
    end
    
    return rmseval, beta, training_data, test_data
end


regression (generic function with 1 method)

In [475]:
function ridge_regression(standardised_data, lambda, trainTestSplitPercentage)
    training_data, test_data = split_data(standardised_data, trainTestSplitPercentage)
    X_train = Matrix(training_data[:, Not([:consommation, :id])])
    y_train = training_data[:, :consommation]
    beta = (X_train'X_train + lambda*I)\X_train'y_train

    rmseval = 0.0
    if trainTestSplitPercentage != 0.0
        X_test = Matrix(test_data[:, Not(:consommation, :id)])
        y_test = test_data[:, :consommation]
        ychap =  X_test * beta
        ychap = (ychap .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        y_test = (y_test .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        # print("ridge: \n")
        # print("r2 = ", rsquared(y_test, ychap), "\n")
        rmseval = rmse(y_test, ychap)
        # print("rmse = ", rmseval , "\n\n")
    end
    return rmseval, beta, training_data, test_data
end

ridge_regression (generic function with 1 method)

In [476]:
# pas tester: (aucune idee si ca marche)

function svd_regression(standardised_data, trainTestSplitPercentage)
    training_data, test_data = split_data(standardised_data, trainTestSplitPercentage)
    X_train = Matrix(training_data[:, Not([:consommation, :id])])
    y_train = training_data[:, :consommation]
    
    U, S, V = svd(X_train)

    # beta = V * (Diagonal(1 ./ S) * U' * y_train)

    beta = V' * Diagonal([s > 1e-10 ? 1/s : 0 for s in S]) * U' * y_train

    rmseval = 0.0
    if trainTestSplitPercentage != 0.0
        X_test = Matrix(test_data[:, Not(:consommation,:id)])
        y_test = test_data[:, :consommation]
        ychap =  X_test * beta
        ychap = (ychap .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        y_test = (y_test .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        # print("regression\n")
        # print("r2 = ", rsquared(y_test, ychap), "\n")
        rmseval = rmse(y_test, ychap)
        # print("rmse = ", rmseval , "\n\n")
    end
    return rmseval,beta, training_data, test_data
end

svd_regression (generic function with 1 method)

In [477]:
trainData = CSV.read("./data/train.csv", DataFrame)
testData = CSV.read("./data/test.csv", DataFrame)
first(testData, 1)

Row,annee,type,nombre_cylindres,cylindree,transmission,boite
,Int64,String31,Int64,String3,String15,String15
1,2014,voiture_moyenne,4,"2,5",traction,manuelle


In [478]:
trainData.consommation = parse.(Float64,replace.(trainData.consommation, "," => "."))
trainData.cylindree = parse.(Float64,replace.(trainData.cylindree, "," => "."))
testData.cylindree = parse.(Float64,replace.(testData.cylindree, "," => "."))


150-element Vector{Float64}:
 2.5
 2.5
 2.5
 2.0
 5.8
 5.0
 5.0
 2.4
 3.5
 5.2
 5.2
 5.9
 2.0
 ⋮
 1.6
 3.3
 5.0
 2.0
 2.0
 1.6
 2.0
 2.0
 3.0
 3.0
 1.5
 2.0

In [479]:
# trainData[!,:volume_gaz] = trainData[!,:nombre_cylindres] .* trainData[!,:cylindree]
# testData[!,:volume_gaz] = testData[!,:nombre_cylindres] .* testData[!,:cylindree]
first(trainData, 1)
COMSOMMATION_MEAN = mean(trainData.consommation)
COMSOMMATION_STD = std(trainData.consommation)


2.139763088813657

In [480]:
trainData[!, :id] = 1:nrow(trainData)
testData[!, :id] = 1:nrow(testData)

1:150

In [481]:
data = getStandardEncodedData(trainData)
nrange = 1000
n = 0
for i in range(0, 1, length=nrange)
    n += regression(data, 0.05)[1]
end
print("average rmse for regression: ", n/nrange, "\n")

n = 0
for i in range(0, 1, length=nrange)
    n += ridge_regression(data, 0.1, 0.05)[1]
end
print("average rmse for ridge: ", n/nrange, "\n")

n = 0
for i in range(0, 1, length=nrange)
    n += svd_regression(data, 0.05)[1]
end
print("average rmse for svd: ", n/nrange, "\n")


anneetypenombre_cylindrescylindreetransmissionboiteconsommationidaverage rmse for regression: 0.9270643573090507
average rmse for ridge: 0.9286763295049841
average rmse for svd: 3.083517359671328


In [515]:


function predict(data, beta)
    return (Matrix(data[:, Not(:consommation, :id)]) * beta)
end

function destandardize(data)
    return (data .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
end

function predict_and_destandardize(data, beta)
    return destandardize(predict(data, beta))
end



function find_mistakes(data_to_predict, beta, col)
    # prediction = predict_and_destandardize(data_to_predict, beta)
    prediction = (Matrix(data_to_predict[:, Not(:consommation, :id)]) * betatrain)
    prediction = (prediction .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
    data_to_predict[!,:consommation] = data_to_predict[!,:consommation] .* COMSOMMATION_STD .+ COMSOMMATION_MEAN

    idtrain = 1:size(prediction, 1)

    # print(data_to_predict)
    base_data_of_predicted = innerjoin(data_to_predict, trainData, on=:id, makeunique=true)

    scatter(idtrain, prediction)
    scatter!(idtrain, base_data_of_predicted[!,:consommation])


    difference = prediction - base_data_of_predicted[!,:consommation]
    base_data_of_predicted[!, :difference] = difference
    diff_cutoff = 1
    high_diff_rows = base_data_of_predicted[abs.(difference) .> diff_cutoff, :]

    grouped_normal = combine(groupby(base_data_of_predicted, col), nrow => :count)

    grouped_high_diff = combine(groupby(high_diff_rows, col), nrow => :count)

    grouped_high_diff[!, :total_diff] = combine(groupby(high_diff_rows, col), :difference => (x -> sum(abs.(x))) => :total_diff)[:, :total_diff]

    max_diff = combine(groupby(high_diff_rows, col), :difference => (x -> maximum(abs.(x))) => :max_diff)

    grouped_high_diff = leftjoin(grouped_high_diff, max_diff, on=col)
    grouped_high_diff[!, :average_diff] = combine(groupby(high_diff_rows, col), :difference => (x -> mean(abs.(x))) => :average_abs_diff)[:, :average_abs_diff]

    percentage_high_diff = leftjoin(grouped_high_diff, grouped_normal, on=col, makeunique=true)
    percentage_high_diff[!, :percentage] = percentage_high_diff[!, :count] ./ percentage_high_diff[!, :count_1] .* 100
    percentage_high_diff = sort(percentage_high_diff, :percentage, rev=true)

    println("Percentage of data by ", col," with difference higher than 1: ")
    println(percentage_high_diff[:, [col, :percentage, :count, :count_1, :total_diff, :max_diff, :average_diff]])
end




find_mistakes (generic function with 2 methods)

In [518]:
rmseval, betatrain, traindata, testpredictdata = regression(data, 0.2)
find_mistakes(testpredictdata, betatrain, :type)

Percentage of data by type with difference higher than 1: 
10×7 DataFrame
 Row │ type                   percentage  count  count_1  total_diff  max_diff  average_diff 
     │ String31               Float64     Int64  Int64?   Float64     Float64?  Float64      
─────┼───────────────────────────────────────────────────────────────────────────────────────
   1 │ voiture_moyenne          100.0         7        7    19.1404    8.06351       2.73434
   2 │ voiture_grande           100.0         1        1     2.4801    2.4801        2.4801
   3 │ voiture_minicompacte      83.3333      5        6    10.0975    2.2976        2.0195
   4 │ voiture_compacte          80.0         8       10    20.8681    4.97282       2.60852
   5 │ VUS_petit                 70.0        14       20    28.9095    5.74291       2.06497
   6 │ voiture_sous_compacte     64.2857      9       14    16.4233    4.79655       1.82481
   7 │ VUS_standard              58.3333      7       12    19.9621    5.73062       2.8

In [488]:
test_data = getStandardEncodedData(testData)
beta = ridge_regression(data, 10, 0.0)[2]
X_test = Matrix(test_data[!,Not(:id)])
ychap =  X_test * beta

ychap = (ychap .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN


# # print(ychap)
id = 1:150
df_pred = DataFrame(id=id, consommation=ychap)


# using Dates
# current_time = Dates.format(now(), "yyyy-mm-dd_HH-MM-SS")
# file_name = "benchmark_" * current_time * ".csv"
# CSV.write("./soumissions_potentielles/" *file_name, df_pred)

anneetypenombre_cylindrescylindreetransmissionboiteid

Row,id,consommation
,Int64,Float64
1,1,9.38172
2,2,9.77834
3,3,10.5324
4,4,9.92742
5,5,12.4458
6,6,12.3094
7,7,11.9128
8,8,10.1939
9,9,11.866


In [ ]:
# for type in unique(trainData.type)
#     println(type)
#     data_type = trainData[trainData.type .== type, :]
#     println(combine(groupby(data_type, :transmission), :consommation => mean, :volume_gaz => mean, nrow => :nrow))
#     println()
# end

In [ ]:
# for type in unique(trainData.type)
#     println(type)
#     data_type = trainData[trainData.type .== type, :]
#     display(plot(x=data_type.volume_gaz, y=data_type.consommation))
#     println()
# end

## Exploration des données